# Event-shopping Likelihood Model

## Obtaining data

#### In this notebook:
* Run (and test) SQL files that make the event data tables
* Run (and test) SQL file that unloads the data into an S3 bucket 

In [1]:
import os
import sys
import json
import glob
import time

import datetime

import numpy as np
import pandas as pd

import logging
from pythonjsonlogger import jsonlogger

In [2]:
lib_path = os.path.abspath(os.path.join('..','lib'))
sys.path.append(lib_path)
import redshift_utils as rs
import s3_utils as s3

{"asctime": "2018-03-08 16:39:04,737", "levelname": "INFO", "message": "Starting job"}
{"asctime": "2018-03-08 16:39:04,738", "levelname": "INFO", "message": "Starting job"}


In [3]:
### Set up logging 
formatter = jsonlogger.JsonFormatter('%(asctime)s %(levelname)s %(message)s')
logHandler = logging.StreamHandler()
logHandler.setFormatter(formatter)
logger = logging.getLogger('events_unload_logger')
logger.propagate = False
logger.addHandler(logHandler)
logger.setLevel(logging.INFO)

logger.info('Starting job')

{"asctime": "2018-03-08 16:39:05,794", "levelname": "INFO", "message": "Starting job"}


In [4]:
with open('../json_and_txt/mktg_events.json','r') as f:
    mktg_events = json.load(f)#, encoding='ascii')

In [5]:
with open('../json_and_txt/event_dates_select.txt','r') as f:
    event_dates_str = f.read()

In [6]:
key_event = 'anniversary_public_event'
key_short_event = dict([(d['event'],d['short_event']) for d in mktg_events])[key_event]
key_year = 2017

In [10]:
sql_path = os.path.join('..','sql')
suffix = ''#'_test'
#test = True

In [11]:
environment = 'local'
bucket = 'liveramp-testing'
s3_path = 'event_propensity/temp_data/'
handle = 'ep_{0}_{1}_{2}_'.format(key_short_event, key_year, time.strftime('%Y%m%d')) 

#### ~~Run ZZ_create_events_tables.sql~~ Not necessary anymore

In [10]:
#sql_file = 'ZZ_create_events_tables.sql'
#start = time.time()
#logger.info('Running SQL file {}...'.format(sql_file))
#sql = rs.read_sql_file(os.path.join(sql_path, sql_file))
#rs.execute_rs_query(sql)
#end = time.time()
#logger.info('Running SQL required {}s'.format(round(end - start, 3)))

#### Run XX_check_persona_event_dis.sql to check distribution of customer personas in shopping events

In [11]:
event_dates_txt = "\nunion\n\n".join(map(lambda s: event_dates_str.format(suffix, **s), mktg_events))

In [12]:
%%time

sql_file = 'XX_check_persona_event_dist.sql'
start_yr = 2013
end_yr = 2017
dl_path = os.path.join('temp','downloads')
distfile = 'event_shoppers_by_persona.csv'
sql = rs.read_sql_file(os.path.join(sql_path, sql_file))
sql = sql.format(suffix, start_yr, end_yr, event_dates_txt)
rs.execute_rs_query(sql, return_csv = True, csvfilename = os.path.join(dl_path, distfile), delimiter = ',')


#### Run 00_get_event_span.sql

In [12]:
sql_file = '00_get_event_span.sql'
start = time.time()
logger.info('Running SQL file {}...'.format(sql_file))
sql = rs.read_sql_file(os.path.join(sql_path, sql_file))
sql = sql.format(suffix, key_event, key_year)
rows, header = rs.execute_rs_query(sql, return_data=True)
end = time.time()
logger.info('Running SQL required {}s'.format(round(end - start, 3)))

{"asctime": "2018-03-08 16:41:55,757", "levelname": "INFO", "message": "Running SQL file 00_get_event_span.sql..."}
{"asctime": "2018-03-08 16:42:07,456", "levelname": "INFO", "message": "Running SQL required 11.699s"}


In [13]:
start_dt, end_dt = rows[0]
print start_dt, end_dt

2017-07-21 2017-08-06


#### Run 01_unload_data.sql

In [14]:
event_dates_txt = "\nunion\n\n".join(map(lambda s: event_dates_str.format(suffix, **s), mktg_events))

In [26]:
feature_end_dt = start_dt - datetime.timedelta(days=30)

In [28]:
sql_file = '01_unload_data.sql'
start = time.time()
logger.info('Creating and unloading data...')
creds = s3.get_temp_creds(environment=environment, profile_name='default')
sql = rs.read_sql_file(os.path.join(sql_path, sql_file))
sql = sql.format(suffix, start_dt, end_dt, feature_end_dt, event_dates_txt, bucket, s3_path, handle, creds)
rs.execute_rs_query(sql)
end = time.time()
logger.info('Creating and unloading data required {}s'.format(round(end - start, 3)))

{"asctime": "2018-03-08 16:45:14,135", "levelname": "INFO", "message": "Creating and unloading data..."}
{"asctime": "2018-03-08 17:02:53,734", "levelname": "INFO", "message": "Creating and unloading data required 1059.598s"}


#### Download result file from S3 bucket

In [29]:
#handle = 'ep_{0}_{1}_{2}_'.format(key_short_event, key_year, time.strftime('%Y%m%d')) 
dl_path = os.path.join('temp','downloads')
filename = handle + '000.gz'

In [30]:
start = time.time()
logger.info('Downloading data...')
s3.download_file_from_s3(bucket, s3_path, filename, filepath=dl_path, environment=environment, profile_name='default')
end = time.time()
logger.info('Downloading data required {}s'.format(round(end - start, 3)))

{"asctime": "2018-03-08 17:05:38,186", "levelname": "INFO", "message": "Downloading data..."}
{"asctime": "2018-03-08 17:05:38,439", "levelname": "INFO", "message": "S3_path + filename = event_propensity/temp_data/ep_anniversary_public_2017_20180308_000.gz"}
{"asctime": "2018-03-08 17:05:38,440", "levelname": "INFO", "message": "Local filepath + filename = temp/downloads/ep_anniversary_public_2017_20180308_000.gz"}
{"asctime": "2018-03-08 17:07:48,309", "levelname": "INFO", "message": "Downloading data required 130.123s"}
